In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import requests
import json
from pprint import pprint
from config import openWeatherKey
import gmaps
from THconfig import g_key
gmaps.configure(api_key=g_key)

In [6]:
disasterdataCSV = "Resources/disaster_data.csv"
Data_disaster_df = pd.read_csv(disasterdataCSV)
Data_disaster_df

,Dis No,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Entry Criteria,...,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI
0,2000-0420-CAN,2000,420,Natural,Meteorological,Storm,Convective storm,Tornado,NaN,Kill,...,14.0,11.0,140.0,NaN,700.0,840.0,NaN,10000.0,13000.0,67.355759
1,2001-0052-CAN,2001,52,Natural,Meteorological,Storm,NaN,NaN,NaN,Kill,...,10.0,11.0,NaN,NaN,30.0,30.0,NaN,NaN,NaN,69.259340
2,2000-0080-USA,2000,80,Natural,Hydrological,Flood,Riverine flood,NaN,NaN,Affect,...,22.0,3.0,NaN,231.0,NaN,231.0,NaN,NaN,NaN,67.355759
3,2000-0500-USA,2000,500,Natural,Hydrological,Flood,Riverine flood,NaN,NaN,Affect,...,14.0,NaN,NaN,175.0,NaN,175.0,NaN,NaN,166000.0,67.355759
4,2000-0919-USA,2000,919,Natural,Meteorological,Storm,NaN,NaN,NaN,SigDam,...,25.0,4.0,NaN,NaN,NaN,NaN,NaN,280000.0,350000.0,67.355759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,2019-0307-USA,2019,307,Natural,Hydrological,Flood,NaN,NaN,NaN,Affected,...,9.0,3.0,NaN,600.0,NaN,600.0,NaN,NaN,NaN,100.000000
533,2020-0148-USA,2020,148,Natural,Meteorological,Storm,Convective storm,Tornado,NaN,Kill,...,14.0,38.0,200.0,NaN,NaN,200.0,NaN,NaN,1000000.0,NaN
534,2020-0011-USA,2020,11,Natural,Meteorological,Storm,Convective storm,Severe storm,NaN,Kill,...,12.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,1200000.0,NaN
535,2020-0082-USA,2020,82,Natural,Meteorological,Storm,Convective storm,Tornado,NaN,Waiting,...,5.0,25.0,300.0,12000.0,NaN,12300.0,NaN,NaN,1100000.0,NaN


In [7]:
weatherDisasterSouth_EastCSV = "output_data/NEW_coordinates.csv"
wDisaster_df = pd.read_csv(weatherDisasterSouth_EastCSV)
wDisaster_df

,Unnamed: 0,Unnamed: 0.1,Location,Dis No,Lat,Long
0,0,0,Kentucky,2000-0080-USA,34.61,-92.66
1,1,2,Morris,2000-0500-USA,40.83,-74.53
2,2,4,Alabama,2000-0919-USA,32.75,-86.75
3,3,5,Georgia,2000-0919-USA,32.75,-83.50
4,4,6,Louisiana,2000-0919-USA,31.00,-92.00
...,...,...,...,...,...,...
2856,2856,4016,Kentucky,2020-0082-USA,34.61,-92.66
2857,2857,4017,Missouri,2020-0082-USA,38.25,-92.50
2858,2858,4018,Mississippi,2020-0082-USA,32.75,-89.75
2859,2859,4019,Georgia,2020-0082-USA,32.75,-83.50


In [8]:
Full_Data_list_df = pd.merge(wDisaster_df, Data_disaster_df, on="Dis No", how="left")
# Full_Data_list_df

In [9]:
SE_wDisaster_df = wDisaster_df.loc[(wDisaster_df["Long"]>-90) & (wDisaster_df["Lat"]<38),["Location","Dis No","Lat", "Long"]]
SE_wDisaster_df

,Location,Dis No,Lat,Long
2,Alabama,2000-0919-USA,32.75,-86.75
3,Georgia,2000-0919-USA,32.75,-83.50
5,North Carolina,2000-0919-USA,35.50,-80.00
6,South Carolina,2000-0919-USA,34.00,-81.00
7,Tennessee,2000-0919-USA,35.75,-86.25
...,...,...,...,...
2852,Mississippi,2020-0011-USA,32.75,-89.75
2853,Alabama,2020-0011-USA,32.75,-86.75
2854,Tennessee,2020-0011-USA,35.75,-86.25
2858,Mississippi,2020-0082-USA,32.75,-89.75


In [10]:
len(SE_wDisaster_df["Location"].unique())

98

In [11]:
SE_Data_list_df = pd.merge(SE_wDisaster_df, Data_disaster_df, on="Dis No", how="left").rename(columns={"Location_x": "Location"})
SE_Data_list_df

,Location,Dis No,Lat,Long,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,...,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI
0,Alabama,2000-0919-USA,32.75,-86.75,2000,919,Natural,Meteorological,Storm,NaN,...,25.0,4.0,NaN,NaN,NaN,NaN,NaN,280000.0,350000.0,67.355759
1,Georgia,2000-0919-USA,32.75,-83.50,2000,919,Natural,Meteorological,Storm,NaN,...,25.0,4.0,NaN,NaN,NaN,NaN,NaN,280000.0,350000.0,67.355759
2,North Carolina,2000-0919-USA,35.50,-80.00,2000,919,Natural,Meteorological,Storm,NaN,...,25.0,4.0,NaN,NaN,NaN,NaN,NaN,280000.0,350000.0,67.355759
3,South Carolina,2000-0919-USA,34.00,-81.00,2000,919,Natural,Meteorological,Storm,NaN,...,25.0,4.0,NaN,NaN,NaN,NaN,NaN,280000.0,350000.0,67.355759
4,Tennessee,2000-0919-USA,35.75,-86.25,2000,919,Natural,Meteorological,Storm,NaN,...,25.0,4.0,NaN,NaN,NaN,NaN,NaN,280000.0,350000.0,67.355759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,Mississippi,2020-0011-USA,32.75,-89.75,2020,11,Natural,Meteorological,Storm,Convective storm,...,12.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,1200000.0,NaN
575,Alabama,2020-0011-USA,32.75,-86.75,2020,11,Natural,Meteorological,Storm,Convective storm,...,12.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,1200000.0,NaN
576,Tennessee,2020-0011-USA,35.75,-86.25,2020,11,Natural,Meteorological,Storm,Convective storm,...,12.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,1200000.0,NaN
577,Mississippi,2020-0082-USA,32.75,-89.75,2020,82,Natural,Meteorological,Storm,Convective storm,...,5.0,25.0,300.0,12000.0,NaN,12300.0,NaN,NaN,1100000.0,NaN


In [12]:
SE_Data_list_df["Disaster Type"].unique()

array(['Storm', 'Wildfire', 'Drought', 'Flood', 'Epidemic',
       'Extreme temperature'], dtype=object)

In [18]:
# SE_Data_list_df.groupby(["Disaster Type", "Disaster Subtype"]).count()

In [14]:
trimmed_df = SE_Data_list_df.drop_duplicates(subset= "Dis No", keep="first")
trimmed_df

,Location,Dis No,Lat,Long,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,...,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI
0,Alabama,2000-0919-USA,32.75,-86.75,2000,919,Natural,Meteorological,Storm,NaN,...,25.0,4.0,NaN,NaN,NaN,NaN,NaN,280000.0,350000.0,67.355759
6,Bullock,2000-0067-USA,32.07,-85.70,2000,67,Natural,Meteorological,Storm,Convective storm,...,14.0,22.0,100.0,900.0,NaN,1000.0,NaN,NaN,NaN,67.355759
8,Columbia,2000-0338-USA,34.00,-81.03,2000,338,Natural,Meteorological,Storm,Convective storm,...,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28500.0,67.355759
9,Anderson,2000-0360-USA,34.53,-82.65,2000,360,Natural,Meteorological,Storm,Convective storm,...,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.355759
14,Florida,2000-0937-USA,28.75,-82.50,2000,937,Natural,Meteorological,Storm,Convective storm,...,17.0,12.0,75.0,NaN,NaN,75.0,NaN,200500.0,500000.0,67.355759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,Mississippi,2019-0335-USA,32.75,-89.75,2019,335,Natural,Meteorological,Storm,Tropical cyclone,...,15.0,1.0,NaN,NaN,NaN,NaN,NaN,300000.0,600000.0,100.000000
563,Florida,2019-0412-USA,28.75,-82.50,2019,412,Natural,Meteorological,Storm,Tropical cyclone,...,6.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,1200000.0,100.000000
568,Mississippi,2020-0148-USA,32.75,-89.75,2020,148,Natural,Meteorological,Storm,Convective storm,...,14.0,38.0,200.0,NaN,NaN,200.0,NaN,NaN,1000000.0,NaN
574,Mississippi,2020-0011-USA,32.75,-89.75,2020,11,Natural,Meteorological,Storm,Convective storm,...,12.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,1200000.0,NaN


In [16]:
Disastercount_df = trimmed_df.groupby(["Disaster Type", "Disaster Subtype"]).agg({"Location":"count","Total Damages ('000 US$)":"sum"})
Disastercount_df

Location  \
Disaster Type       Disaster Subtype                             
Drought             Drought                                  2   
Epidemic            Viral disease                            2   
Extreme temperature Cold wave                                1   
                    Heat wave                                1   
                    Severe winter conditions                 1   
Flood               Flash flood                              1   
                    Riverine flood                          17   
Storm               Convective storm                        92   
                    Tropical cyclone                        33   
Wildfire            Forest fire                              6   
                    Land fire (Brush, Bush, Pasture)         2   

                                                      Total Damages ('000 US$)  
Disaster Type       Disaster Subtype                                            
Drought             Drought                                          1400000.0  
Epidemic            Viral disease                                          0.0  
Extreme temperature Cold wave                                        2500000.0  
                    Heat wave                                              0.0  
                    Severe winter conditions                          200000.0  
Flood               Flash flood                                      2700000.0  
                    Riverine flood                                   7633000.0  
Storm               Convective storm                               109587400.0  
                    Tropical cyclone                               477895360.0  
Wildfire            Forest fire                                      2000000.0  
                    Land fire (Brush, Bush, Pasture)                13400000.0

In [17]:
trimmed_df.groupby(["Disaster Type"]).agg({"Location":"count","Total Damages ('000 US$)":"sum"})

,Location,Total Damages ('000 US$)
Disaster Type,,
Drought,2,1400000.0
Epidemic,2,0.0
Extreme temperature,3,2700000.0
Flood,22,21033000.0
Storm,134,593433260.0
Wildfire,9,20600000.0


In [34]:
locations = trimmed_df[["Lat", "Long"]]
ratings = trimmed_df["Total Damages ('000 US$)"].astype(float)
max_damages = max(trimmed_df["Total Damages ('000 US$)"])
print(max_damages)

125000000.0


In [35]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = ratings, dissipating=False, max_intensity=max_damages,
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
trimmed_year_df = trimmed_df.groupby(["Year", ).sum()
trimmed_year_df.columns

Index(['Lat', 'Long', 'Seq', 'OFDA Response', 'Aid Contribution',
       'Dis Mag Value', 'Latitude', 'Longitude', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day', 'Total Deaths',
       'No Injured', 'No Affected', 'No Homeless', 'Total Affected',
       'Reconstruction Costs ('000 US$)', 'Insured Damages ('000 US$)',
       'Total Damages ('000 US$)', 'CPI'],
      dtype='object')